### FastFeatureDetector 특징 검출 1

In [4]:
# 0901.py
import cv2
import numpy as np
 
src = cv2.imread('c:/data/chessBoard.jpg')
gray= cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)

#1 꼭짓점 검출
##fastF = cv2.FastFeatureDetector_create()
# fastF =cv2.FastFeatureDetector.create()
fastF =cv2.FastFeatureDetector.create(threshold=30) # 100
kp = fastF.detect(gray) 
dst = cv2.drawKeypoints(gray, kp, None, color=(0,0,255))
print('len(kp)=', len(kp))
cv2.imshow('dst',  dst)

#2 주변 값을 억제하지 않은 경우
fastF.setNonmaxSuppression(False)
kp2 = fastF.detect(gray)
dst2 = cv2.drawKeypoints(src, kp2, None, color=(0,0,255))
print('len(kp2)=', len(kp2))
cv2.imshow('dst2',  dst2)

#3
dst3 = src.copy()
points = cv2.KeyPoint_convert(kp)
points = np.int32(points)

for cx, cy in points:
    cv2.circle(dst3, (cx, cy), 3, color=(255, 0, 0), thickness=1)
cv2.imshow('dst3',  dst3)
cv2.waitKey()
cv2.destroyAllWindows()


len(kp)= 98
len(kp2)= 867


### 9.3 MSER GRAY or Color 에서 주변보다 더 밝거나 더 어두운 여역으로 임계값의 범위에서 안정적인 영역을 특징으로 검출

In [6]:
# 0903.py
import cv2
import numpy as np
 
src = cv2.imread('c:/data/chessBoard.jpg')
gray= cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)

#1 중심점 검출
mserF = cv2.MSER_create(10)  # cv2.MSER.create(10)
kp= mserF.detect(gray)
print('len(kp)=', len(kp))
dst = cv2.drawKeypoints(gray, kp, None, color=(0,0,255))
cv2.imshow('dst',  dst)

#2 특징 영역의 좌표 계산 후 출력
dst2 = dst.copy()
regions, bboxes = mserF.detectRegions(gray)
hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions]
cv2.polylines(dst2, hulls, True, (0, 255, 0))
cv2.imshow('dst2',  dst2)

#3 각 영역의 좌표를 통해 파란색 타원과 초록색 사각형 그리기
dst3 = dst.copy()
for i, pts in enumerate(regions):
    box = cv2.fitEllipse(pts)
    cv2.ellipse(dst3, box,  (255,0,0),1) # 타원
    x, y, w, h = bboxes[i]
    cv2.rectangle(dst3, (x, y), (x+w, y+h), (0,255,0)) # 사각형
cv2.imshow('dst3',  dst3)
cv2.waitKey()
cv2.destroyAllWindows()

len(kp)= 202


### 9.6 ORB 특징 검출 및 디스크립터

In [5]:
# 0906.py
import cv2
import numpy as np
#1
def distance(f1, f2):    
    x1, y1 = f1.pt
    x2, y2 = f2.pt
    return np.sqrt((x2 - x1)**2+ (y2 - y1)**2)

def filteringByDistance(kp, distE=0.5):
    size = len(kp)
    mask = np.arange(1,size+1).astype(np.bool8) # all True   
    for i, f1 in enumerate(kp):
        if not mask[i]:
            continue
        else: # True
            for j, f2 in enumerate(kp):
                if i == j:
                    continue
                if distance(f1, f2)<distE:
                    mask[j] = False
    np_kp = np.array(kp)
    return list(np_kp[mask])

#2
src = cv2.imread('c:/data/temp/pad.jpg')
src = cv2.resize(src, (512, 512))
# src = cv2.imread('c:/data/cornerTest.jpg')
##src = cv2.imread('./data/chessBoard.jpg')
gray= cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0.0)

##orbF = cv2.ORB_create()          
orbF = cv2.ORB_create(scoreType=1)
kp= orbF.detect(gray)
print('len(kp)=', len(kp))
dst = cv2.drawKeypoints(gray, kp, None, color=(0,0,255))   
cv2.imshow('dst',  dst)

#3
kp = sorted(kp, key=lambda f: f.response, reverse=True)
filtered_kp = list(filter(lambda f: f.response>50, kp))
filtered_kp = filteringByDistance(kp, 10)
print('len(filtered_kp)=', len(filtered_kp))

kp, des = orbF.compute(gray, filtered_kp)
print('des.shape=', des.shape)
print('des=', des)

#4
dst2 = cv2.drawKeypoints(gray, filtered_kp, None, color=(0,0,255))  
for f in filtered_kp:
    x, y = f.pt
    size = f.size
    rect = ((x, y), (size, size), f.angle)
    box = cv2.boxPoints(rect).astype(np.int32)
    cv2.polylines(dst2, [box], True, (0,255,0), 2)
    cv2.circle(dst2, (round(x), round(y)), round(f.size/2), (255,0,0), 2)
cv2.imshow('dst2',  dst2)
cv2.waitKey()
cv2.destroyAllWindows()

len(kp)= 97
len(filtered_kp)= 27
des.shape= (27, 32)
des= [[ 14  92 102 187  55 243 188 230  29  79  84 183 108  76  54 201 255  51
  151 131 166 180  57 255 103  95 218 131 151  93 189 215]
 [110  18 235 236 164 114  94  82   5 142 223  34  73 189 179 104 217 182
  151 161 140 163  32 136 211 255 238 137 150 250 116  67]
 [149  89 233 199 172  86 239 216 196 192 214  66 113 252 245  98 120 100
   62 251 201 165 127 170 123  40 175  16  53 187 112 100]
 [154  95  46 188   6 132 191 118 132 142 214 167 109 239  51  67 110 241
  159 227 232 167  34 238 243  67 238 129  54 253  53 123]
 [ 34 216 104 136 111 209 154 200 143  69 156 163  44 222 102 196 206  57
  147 193   6   3  42 241  18  84 138 251  23  14  61 223]
 [ 63  11  98 206  93 151 238 112 197  76 221   6 105 200  55 224 120 168
  151 235 204 163  20 220 115 109 162 137  55 190 120 198]
 [ 66 218  56 186  47 193 152 236 143  29 148 163 172 206  98  64 207  19
  191 129 198  11  74 241 208  81 187 251  20  12  61  91]
 [ 28  13  

### 9.8 BRISK 특징 검출 및 디스크립터

In [3]:
# 0908.py
import cv2
import numpy as np

#1
def distance(f1, f2):    
    x1, y1 = f1.pt
    x2, y2 = f2.pt
    return np.sqrt((x2 - x1)**2+ (y2 - y1)**2)

def filteringByDistance(kp, distE=0.5):
    size = len(kp)
    mask = np.arange(1,size+1).astype(np.bool8) # all True   
    for i, f1 in enumerate(kp):
        if not mask[i]:
            continue
        else: # True
            for j, f2 in enumerate(kp):
                if i == j:
                    continue
                if distance(f1, f2)<distE:
                    mask[j] = False
    np_kp = np.array(kp)
    return list(np_kp[mask])
    
#2
src = cv2.imread('c:/data/temp/pad.jpg')
src = cv2.resize(src, (512, 512))
gray= cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0.0)

briskF = cv2.BRISK_create()
kp= briskF.detect(gray)
print('len(kp)=', len(kp))
dst = cv2.drawKeypoints(gray, kp, None, color=(0,0,255))   
cv2.imshow('dst',  dst)

#3
kp = sorted(kp, key=lambda f: f.response, reverse=True)
filtered_kp = list(filter(lambda f: f.response>50, kp))
filtered_kp = filteringByDistance(kp, 10)
print('len(filtered_kp)=', len(filtered_kp))

kp, des = briskF.compute(gray, filtered_kp)
print('des.shape=', des.shape)
print('des=', des)

#4
dst2 = cv2.drawKeypoints(gray, filtered_kp, None, color=(0,0,255))  
for f in filtered_kp:
    x, y = f.pt
    size = f.size
    rect = ((x, y), (size, size), f.angle)
    box = cv2.boxPoints(rect).astype(np.int32)
    cv2.polylines(dst2, [box], True, (0,255,0), 2)
    cv2.circle(dst2, (round(x), round(y)), round(f.size/2), (255,0,0), 2)
cv2.imshow('dst2',  dst2)
cv2.waitKey()
cv2.destroyAllWindows()

len(kp)= 35
len(filtered_kp)= 16
des.shape= (16, 64)
des= [[252 255 239 ... 141 128 137]
 [  0   0   0 ... 199  78 205]
 [240 255 239 ... 141   8 185]
 ...
 [255 157 243 ... 199 207 223]
 [255 255 255 ... 205 156 255]
 [255 191 227 ... 152 153  11]]


### ORB, BRISK 매칭

In [68]:
# 0912.py
import cv2
import numpy as np

#1    
src1 = cv2.imread('c:/data/temp/mouse1.jpg') # 'cup1.jpg'
src2 = cv2.imread('c:/data/temp/mouse3.jpg') # 'cup2.jpg'
src1 = cv2.resize(src1, (300, 300))
src2 = cv2.resize(src2, (300, 300))
img1= cv2.cvtColor(src1,cv2.COLOR_BGR2GRAY)
img2= cv2.cvtColor(src2,cv2.COLOR_BGR2GRAY)

#2-1 ORB의 특징점(kp) 검출, 디스크립터(des) 계산
# orbF   = cv2.ORB_create(nfeatures=100)
# kp1, des1 = orbF.detectAndCompute(img1, None)
# kp2, des2 = orbF.detectAndCompute(img2, None)

#2-2 BRICKF의 특징점(kp) 검출, 디스크립터(des) 계산
briskF = cv2.BRISK_create()
kp1, des1 = briskF.detectAndCompute(img1, None)
kp2, des2 = briskF.detectAndCompute(img2, None)

#3-1 BF 매칭으로 확인
# bf = cv2.BFMatcher_create(cv2.NORM_HAMMING, crossCheck=True)
# matches = bf.match(des1,des2)
	
#3-2 FlannBase 매칭으로 확인
flan = cv2.FlannBasedMatcher_create() 
matches = flan.match(np.float32(des1),np.float32(des2))
	
#4 투영 계산
matches = sorted(matches, key = lambda m: m.distance)
print('len(matches)=', len(matches))
for i, m in enumerate(matches[:3]):
        print('matches[{}]=(queryIdx:{}, trainIdx:{}, distance:{})'.format(
            i, m. queryIdx, m.trainIdx, m.distance))
	
minDist = matches[0].distance
good_matches = list(filter(lambda m: m.distance<5*minDist, matches))
print('len(good_matches)=', len(good_matches))
if len(good_matches) < 5:
    print('sorry, too small good matches')
    exit()
    
dst = cv2.drawMatches(img1,kp1,img2,kp2,good_matches, None, flags=2)
cv2.imshow('dst',  dst)

#5 특징점을 리스트에 저장 후 mask 변환
src1_pts = np.float32([ kp1[m.queryIdx].pt for m in good_matches])
src2_pts = np.float32([ kp2[m.trainIdx].pt for m in good_matches])

H, mask = cv2.findHomography(src1_pts, src2_pts, cv2.RANSAC, 3.0)#cv2.LMEDS
mask_matches = mask.ravel().tolist() # list(mask.flatten())

#6 영상의 모서리 좌표 저장
h,w = img1.shape
pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
pts2 = cv2.perspectiveTransform(pts, H)
src2 = cv2.polylines(src2,[np.int32(pts2)],True,(255,0, 0),2)
        
draw_params=dict(matchColor = (0,255,0), singlePointColor = None,
                 matchesMask = mask_matches,  flags = 2)                 
dst2 = cv2.drawMatches(src1,kp1,src2,kp2, good_matches, None,**draw_params)  
cv2.imshow('dst2',  dst2)
cv2.waitKey()
cv2.destroyAllWindows()

len(matches)= 224
matches[0]=(queryIdx:0, trainIdx:7, distance:288.5220947265625)
matches[1]=(queryIdx:167, trainIdx:7, distance:356.2218322753906)
matches[2]=(queryIdx:56, trainIdx:119, distance:372.7036437988281)
len(good_matches)= 224


### VLAD 사용

In [1]:
import cv2
import numpy as np

def extract_local_descriptors(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Create a SIFT object
    sift = cv2.xfeatures2d.SIFT_create()

    # Detect and compute local descriptors
    keypoints, descriptors = sift.detectAndCompute(gray, None)

    return descriptors
# List of image filenames
path = 'c:/data/temp/'
image_filenames = [ path + 'bottle1.jpg', path + 'bottle2.jpg', path + 'bottle3.jpg', path + 'bottle4.jpg']

# Extract local descriptors from the images
descriptors_list = []
for image_filename in image_filenames:
    image = cv2.imread(image_filename)
    image = cv2.resize(image, (32, 32))
    descriptors = extract_local_descriptors(image)
    descriptors_list.append(descriptors)

# Concatenate the descriptors into a single numpy array
descriptors_concatenated = np.concatenate(descriptors_list)

# Perform k-means clustering to obtain visual words
K = 2  # Number of visual words
term_crit = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
flags = cv2.KMEANS_RANDOM_CENTERS
_, labels, visual_words = cv2.kmeans(descriptors_concatenated, K, None, term_crit, 10, flags)

# Initialize the VLAD vector
vlad_vector = np.zeros(K * descriptors_concatenated.shape[1])

# Compute the residuals and accumulate them to the VLAD vector
for descriptors in descriptors_list:
    _, labels, _ = cv2.kmeans(descriptors, K, None, term_crit, 10, flags)
    for i in range(len(labels)):
        vlad_vector[int(labels[i]) * descriptors_concatenated.shape[1]:(int(labels[i]) + 1) * descriptors_concatenated.shape[1]] += descriptors[i] - visual_words[int(labels[i])]

# L2 normalize the VLAD vector
vlad_vector /= np.linalg.norm(vlad_vector)

print(len(vlad_vector))

256


### vlad 2

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans

def extract_local_descriptors(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Create a SIFT object
    sift = cv2.xfeatures2d.SIFT_create()

    # Detect and compute local descriptors
    keypoints, descriptors = sift.detectAndCompute(gray, None)

    return descriptors

# List of image filenames
path = 'c:/data/temp/'
image_filenames = [ path + 'mouse1.jpg', path + 'mouse2.jpg', path + 'mouse3.jpg', path + 'mouse4.jpg', path + 'mouse5.jpg']


# Extract local descriptors from the images
descriptors_list = []
for image_filename in image_filenames:
    image = cv2.imread(image_filename)
    image = cv2.resize(image, (32, 32))
    descriptors = extract_local_descriptors(image)
    descriptors_list.append(descriptors)

# Concatenate the descriptors into a single numpy array
descriptors_concatenated = np.concatenate(descriptors_list)

# Perform k-means clustering to obtain visual words
num_clusters = 2  # Number of visual words
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(descriptors_concatenated)
visual_words = kmeans.cluster_centers_

# Initialize the VLAD vector
vlad_vector = np.zeros(num_clusters * descriptors_concatenated.shape[1])

# Compute the residuals and accumulate them to the VLAD vector
for descriptors in descriptors_list:
    labels = kmeans.predict(descriptors)
    for i in range(len(labels)):
        vlad_vector[labels[i]*descriptors_concatenated.shape[1]:(labels[i]+1)*descriptors_concatenated.shape[1]] += descriptors[i] - visual_words[labels[i]]

# L2 normalize the VLAD vector
vlad_vector /= np.linalg.norm(vlad_vector)

print(vlad_vector, len(vlad_vector))

### vlad로 디스크립터를 연산한 이미지 검색

In [10]:
import cv2
import numpy as np

imgSize = (300, 300)

def compute_vlad(descriptors, visual_words):
    k = visual_words.shape[0]  # Number of visual words
    d = descriptors.shape[1]   # Dimension of descriptors
    n = descriptors.shape[0]   # Number of descriptors

    # Assign each descriptor to the nearest visual word
    _, label, _ = cv2.kmeans(descriptors, k, None, criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 0.01), attempts=3, flags=cv2.KMEANS_RANDOM_CENTERS)

    # Compute residuals and accumulate them for each visual word
    vlad_vector = np.zeros((k, d), dtype=np.float32)
    for i in range(k):
        # Find descriptors assigned to the current visual word
        assigned_descriptors = descriptors[label.ravel() == i]

        # Subtract the visual word from the assigned descriptors to compute residuals
        residuals = assigned_descriptors - visual_words[i]

        # Accumulate the residuals
        vlad_vector[i] = np.sum(residuals, axis=0)

    # L2 normalize the VLAD vector
    vlad_vector = cv2.normalize(vlad_vector, None, alpha=1, norm_type=cv2.NORM_L2)

    # Flatten the VLAD vector
    vlad_vector = vlad_vector.flatten()

    return vlad_vector

def compute_vlad_descriptor(image_path, sift, visual_words):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, imgSize)
    # Detect keypoints and compute descriptors using SIFT
    keypoints, descriptors = sift.detectAndCompute(image, None)

    # Compute the VLAD descriptor
    if descriptors is not None:
        vlad_descriptor = compute_vlad(descriptors, visual_words)
    else:
        vlad_descriptor = None

    return vlad_descriptor

def search_similar_images(query_image_path, database_image_paths, num_results):
    sift = cv2.xfeatures2d.SIFT_create()

    # Compute VLAD descriptors for database images
    database_descriptors = []
    for image_path in database_image_paths:
        vlad_descriptor = compute_vlad_descriptor(image_path, sift, visual_words)
        if vlad_descriptor is not None:
            database_descriptors.append(vlad_descriptor)

    query_image = cv2.imread(query_image_path, cv2.IMREAD_GRAYSCALE)
    query_image = cv2.resize(query_image, (300, 300))  # Resize query image to 300x300
    _, query_descriptors = sift.detectAndCompute(query_image, None)

    # Create FLANN-based matcher
    flann = cv2.FlannBasedMatcher()

    # Iterate over the database images
    similar_images = []
    for image_path in database_image_paths:
        # Compute descriptors for the database image
        database_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        database_image = cv2.resize(database_image, (300, 300))  # Resize database image to 300x300
        _, database_descriptors = sift.detectAndCompute(database_image, None)

        # Match descriptors using FLANN-based matcher
        matches = flann.knnMatch(query_descriptors, database_descriptors, k=2)

        # Apply ratio test to filter good matches
        good_matches = []
        for m, n in matches:
            if m.distance < 0.7 * n.distance:
                good_matches.append(m)

        # Store the number of good matches for the image
        num_good_matches = len(good_matches)
        similar_images.append((image_path, num_good_matches))

    # Sort the images based on the number of good matches
    similar_images.sort(key=lambda x: x[1], reverse=True)

    # Retrieve the top similar images
    similar_images = [image_path for image_path, _ in similar_images[:num_results]]

    return similar_images

# Set the path to the query image
path = 'c:/data/temp/'
query_image_path = path + 'mouse5.jpg' # 검색 이미지

# 데이터 이미지
database_image_paths = [path + 'mouse1.jpg', path + 'mouse2.jpg', path + 'mouse3.jpg', path + 'mouse4.jpg', path + 'mouse7.jpg', 
                        path + 'bottle1.jpg', path + 'bottle2.jpg', path + 'bottle3.jpg', path + 'bottle4.jpg', path + 'bottle5.jpg',
                        path + 'flower1.jpg', path + 'flower2.jpg', path + 'flower3.jpg', path + 'flower4.jpg', path + 'flower5.jpg']

# Set the number of similar images to retrieve
num_results = 7

# Perform the image search
similar_images = search_similar_images(query_image_path, database_image_paths, num_results)

# Display the similar images
for image_path in similar_images:
    image = cv2.imread(image_path)
    image = cv2.resize(image, imgSize)
    cv2.imshow('Similar Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
import cv2
import numpy as np
import faiss

imgSize = (300, 300)

def extract_sift_features(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, imgSize)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)

    return descriptors


def perform_kmeans(descriptors, num_clusters=16):
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 0.1)
    _, labels, cluster_centers = cv2.kmeans(descriptors, num_clusters, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    return cluster_centers


def compute_vlad(descriptors, cluster_centers):
    vlad_vector = np.zeros((cluster_centers.shape[0], cluster_centers.shape[1]), dtype=np.float32)
    index = faiss.IndexFlatIP(cluster_centers.shape[1])
    index.add(cluster_centers)
    _, indices = index.search(descriptors.reshape(-1, cluster_centers.shape[1]), 1)
    for i, visual_word_index in enumerate(indices):
        vlad_vector[visual_word_index] += descriptors[i] - cluster_centers[visual_word_index]

    vlad_vector = vlad_vector.flatten()
    vlad_vector /= np.linalg.norm(vlad_vector)

    return vlad_vector


def search_similar_images(query_image_path, database_image_paths, num_results=5):
    # Extract SIFT features from the query image
    query_descriptors = extract_sift_features(query_image_path)

    # Perform K-means clustering on the database descriptors
    database_descriptors = []
    for image_path in database_image_paths:
        descriptors = extract_sift_features(image_path)
        database_descriptors.extend(descriptors)

    database_descriptors = np.array(database_descriptors)
    cluster_centers = perform_kmeans(database_descriptors)

    # Compute VLAD vectors for the query descriptors
    query_vlad_vector = compute_vlad(query_descriptors, cluster_centers)

    # Initialize Faiss index
    index = faiss.IndexFlatIP(cluster_centers.shape[1])

    # Compute VLAD vectors for the database descriptors
    database_vlad_vectors = []
    for descriptors in database_descriptors:
        vlad_vector = compute_vlad(descriptors, cluster_centers)
        database_vlad_vectors.append(vlad_vector)

    database_vlad_vectors = np.array(database_vlad_vectors)

    # Add database vectors to Faiss index
    index.add(database_vlad_vectors)

    # Search for similar images
    _, indices = index.search(np.array([query_vlad_vector]), num_results)
    similar_images = [database_image_paths[i] for i in indices[0]]

    return similar_images


# Set the path to the query image
path = 'c:/data/temp/'
query_image_path = path + 'mouse5.jpg' # 검색 이미지

# 데이터 이미지
database_image_paths = [path + 'mouse1.jpg', path + 'mouse2.jpg', path + 'mouse3.jpg', path + 'mouse4.jpg', path + 'mouse7.jpg', 
                        path + 'bottle1.jpg', path + 'bottle2.jpg', path + 'bottle3.jpg', path + 'bottle4.jpg', path + 'bottle5.jpg',
                        path + 'flower1.jpg', path + 'flower2.jpg', path + 'flower3.jpg', path + 'flower4.jpg', path + 'flower5.jpg']

# Set the number of similar images to retrieve
num_results = 7

# Perform the image search
similar_images = search_similar_images(query_image_path, database_image_paths, num_results)

# Display the similar images
for image_path in similar_images:
    image = cv2.imread(image_path)
    image = cv2.resize(image, imgSize)
    cv2.imshow('Similar Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


AssertionError: 

### 이미지 검색, 잘 됨

In [52]:
import cv2
import numpy as np

imgSize = (64, 64)

# num_results 출력할 비슷한 이미지 개수
def detect_similar_images(query_image_path, database_image_paths, num_results=5):
    # Load the query image
    query_image = cv2.imread(query_image_path, cv2.IMREAD_GRAYSCALE)
    query_image = cv2.resize(query_image, imgSize)
    
    # Create the feature detector (SIFT)
    sift = cv2.xfeatures2d.SIFT_create()

    # Detect keypoints and compute descriptors for the query image
    query_keypoints, query_descriptors = sift.detectAndCompute(query_image, None)
    
    # Create a brute-force matcher
    bf = cv2.BFMatcher()

    # List to store the similarity scores and corresponding image paths
    similarity_scores = []
    similar_image_paths = []

    # Iterate over the database images
    for image_path in database_image_paths:
        # Load the database image
        database_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        database_image = cv2.resize(database_image, imgSize)
        # Detect keypoints and compute descriptors for the database image
        database_keypoints, database_descriptors = sift.detectAndCompute(database_image, None)

        # Match the descriptors between the query and database images
        matches = bf.match(query_descriptors, database_descriptors)

        # Calculate the similarity score as the total distance of matched keypoints
        similarity_score = sum([match.distance for match in matches])

        # Store the similarity score and image path
        similarity_scores.append(similarity_score)
        similar_image_paths.append(image_path)

    # Sort the images based on similarity scores (lower score indicates higher similarity)
    sorted_indices = np.argsort(similarity_scores)

    # Retrieve the top similar images
    similar_images = []
    for i in range(num_results):
        image_path = similar_image_paths[sorted_indices[i]]
        image = cv2.imread(image_path)
        similar_images.append(image)

    return similar_images

# Set the path to the query image
path = 'c:/data/temp/'
query_image_path = path + 'bottle7.jpg' # 검색 이미지

# 데이터 이미지
database_image_paths = [path + 'mouse1.jpg', path + 'mouse2.jpg', path + 'mouse3.jpg', path + 'mouse4.jpg', path + 'mouse7.jpg', 
                        path + 'bottle1.jpg', path + 'bottle2.jpg', path + 'bottle3.jpg', path + 'bottle4.jpg', path + 'bottle5.jpg']

# Detect similar images
similar_images = detect_similar_images(query_image_path, database_image_paths)

# Display the similar images
for i in range(len(similar_images)):
    image = cv2.resize(similar_images[i], (300, 300))
    cv2.imshow('Similar Image'+str(i), image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### faiss 추가 버전

In [61]:
import cv2
import numpy as np
import faiss

imgSize = (64, 64)

def extract_local_descriptors(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return keypoints, descriptors

def detect_similar_images(query_image_path, database_image_paths, num_results=5):
    # Load the query image
    query_image = cv2.imread(query_image_path)
    query_image = cv2.resize(query_image, imgSize)
    query_keypoints, query_descriptors = extract_local_descriptors(query_image)

    # Create an index for Faiss
    d = query_descriptors.shape[1]  # Dimension of the descriptors
    index = faiss.IndexFlatIP(d)    # Inner product (dot product) distance measure

    # Add the query descriptor to the index
    index.add(np.float32(query_descriptors))

    # List to store the similarity scores and corresponding image paths
    similarity_scores = []
    similar_image_paths = []

    # Iterate over the database images
    for image_path in database_image_paths:
        # Load the database image
        database_image = cv2.imread(image_path)
        database_image = cv2.resize(database_image, imgSize)
        database_keypoints, database_descriptors = extract_local_descriptors(database_image)

        # Perform a k-nearest neighbor search using Faiss
        k = 1  # Number of nearest neighbors to retrieve
        _, indices = index.search(np.float32(database_descriptors), k)

        # Check if indices are within the valid range
        valid_indices = indices[indices < len(database_descriptors)]
        if len(valid_indices) == 0:
            continue

        # Reshape the database descriptors to remove the extra dimension
        database_descriptors = np.squeeze(database_descriptors[valid_indices])

        # Compute the similarity score as the inner product (dot product) between query and database descriptors
        similarity_score = np.dot(query_descriptors, database_descriptors.T)

        # Store the similarity score and image path
        similarity_scores.append(similarity_score)
        similar_image_paths.append(image_path)

    # Sort the images based on similarity scores (higher score indicates higher similarity)
    sorted_indices = np.argsort(similarity_scores)[::-1]

    # Retrieve the top similar images
    similar_images = []
    for i in range(num_results):
        if i >= len(sorted_indices):
            break
        image_path = similar_image_paths[sorted_indices[i]]
        image = cv2.imread(image_path)
        similar_images.append(image)

    return similar_images

# Set the path to the query image
path = 'c:/data/temp/'
query_image_path = path + 'bottle7.jpg' # 검색 이미지

# 데이터 이미지
database_image_paths = [ path + 'mouse1.jpg', path + 'mouse2.jpg', path + 'mouse3.jpg', path + 'mouse4.jpg', path + 'mouse5.jpg', 
                        path + 'bottle1.jpg', path + 'bottle2.jpg', path + 'bottle3.jpg', path + 'bottle4.jpg', path + 'bottle5.jpg']

# Detect similar images
similar_images = detect_similar_images(query_image_path, database_image_paths, num_results=5)

# Display the similar images
for image in similar_images:
    cv2.imshow('Similar Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


ValueError: could not broadcast input array from shape (22,12) into shape (22,)